# Build Python Models to Predict Auto Loan Default

- XGBoost:  Serialize with Pickle and ONNX
- Tensorflow: Serialize with H5

In [1]:
import csv
import pandas as pd
import numpy as np
import os, shutil # used to create necessary folders
import pickle
import json
import sklearn
import tensorflow as tf
from xgboost import XGBClassifier 
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
 
 


2022-06-10 16:43:23.018804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-10 16:43:23.018831: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
auto = pd.read_csv('Data_orig/AutoLoan.csv')

auto.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)

auto

 

,Loan_ID,BAD,LOAN,LOANDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,772418,1,1100,25860.0000,39025.000000,CarImp,Other,10.500000,0.00000,0.000000,94.366667,1.000000,9.000000,33.779915
1,477724,1,1300,70053.0000,68400.000000,CarImp,Other,7.000000,0.00000,2.000000,121.833333,0.000000,14.000000,33.779915
2,150746,1,1500,13500.0000,16700.000000,CarImp,Other,4.000000,0.00000,0.000000,149.466667,1.000000,10.000000,33.779915
3,108584,1,1500,73760.8172,101776.048741,NaN,NaN,8.922268,0.25457,0.449442,179.766275,1.186055,21.296096,33.779915
4,321534,0,1700,97800.0000,112000.000000,CarImp,Office,3.000000,0.00000,0.000000,93.333333,0.000000,14.000000,33.779915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,346389,0,88900,57264.0000,90185.000000,DebtCon,Other,16.000000,0.00000,0.000000,221.808718,0.000000,16.000000,36.112347
5956,255372,0,89000,54576.0000,92937.000000,DebtCon,Other,16.000000,0.00000,0.000000,208.692070,0.000000,15.000000,35.859971
5957,668928,0,89200,54045.0000,92924.000000,DebtCon,Other,15.000000,0.00000,0.000000,212.279697,0.000000,15.000000,35.556590
5958,233047,0,89800,50370.0000,91861.000000,DebtCon,Other,14.000000,0.00000,0.000000,213.892709,0.000000,16.000000,34.340882


In [3]:
auto.dtypes
 

Loan_ID      int64
BAD          int64
LOAN         int64
LOANDUE    float64
VALUE      float64
REASON      object
JOB         object
YOJ        float64
DEROG      float64
DELINQ     float64
CLAGE      float64
NINQ       float64
CLNO       float64
DEBTINC    float64
dtype: object

In [4]:

 
target         = ["BAD"]
numeric_inputs = ['LOAN', 'LOANDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ','CLNO', 'DEBTINC']
class_inputs   = ['REASON', 'JOB']
 
impute_values = auto[numeric_inputs].mean()
pickle.dump(impute_values, open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle','wb'))
pickle.dump(impute_values, open('Data_orig/autoloan_impute.pickle','wb'))
   
    
auto           =auto.fillna(impute_values)
auto.REASON.replace(np.NaN,'CarImp',regex = True, inplace=True)
auto.JOB.replace(np.nan,'Other',regex = True, inplace=True)

 
x              = auto.drop(['BAD', 'Loan_ID'], axis=1)
y              = auto[target].astype("category")


 
impute_values
 

LOAN        18607.969799
LOANDUE     73760.817200
VALUE      101776.048741
YOJ             8.922268
DEROG           0.254570
DELINQ          0.449442
CLAGE         179.766275
NINQ            1.186055
CLNO           21.296096
DEBTINC        33.779915
dtype: float64

In [5]:
auto2 =auto.head()
auto2.drop(['BAD'], axis=1, inplace=True)
auto2.to_csv('Data_orig/autoloan2_ds2_test.csv', index=False)
 
 
    

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Split Dataset Into Training and Test Set

In [6]:
# Split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test

print("train size: " + str(x_train.shape))
print("test size: " + str(x_test.shape))
print("train sample")

 
 
 
 

train size: (4172, 12)
test size: (1788, 12)
train sample


# Custom preprocessing for categorical inputs to use in XGB model

In [7]:
 
ohe = OneHotEncoder(sparse = False, handle_unknown="ignore")
enc = ohe.fit_transform(x_train[class_inputs])
enc2 = ohe.transform(x_test[class_inputs])

 
x_train[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc, index=x_train.index)
x_train.drop(class_inputs,1,inplace=True)

x_test[ohe.get_feature_names(class_inputs)] = pd.DataFrame(enc2, index=x_test.index)
x_test.drop(class_inputs,1,inplace=True)


print(x_train.shape)
print(x_test.shape)


 
 


(4172, 18)
(1788, 18)


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/3776711327.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train.drop(class_inputs,1,inplace=True)
/tmp/ipykernel_30379/3776711327.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_test.drop(class_inputs,1,inplace=True)


In [8]:
#Pickle the one hot encoding
pickle.dump(ohe, open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle', 'wb'))
pickle.dump(ohe, open('Data_orig/autoloan_encoder.pickle', 'wb'))

# Standardize Inputs with Standard Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#scaler = MinMaxScaler()
# fit scaler on the training dataset
#scaler.fit(X_train)
#X_train_scaled = scaler.transform(X_train)

pickle.dump(scaler, open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'wb'))
pickle.dump(scaler, open('Data_orig/autoloan_scaler.pickle', 'wb'))

type(x_train)

numpy.ndarray

# Train the XGBoost Model

In [10]:
# Create XGBoost classifer object

 
# x_train_array = x_train.values
# y_train_array = y_train.values
    
booster = xgb.XGBClassifier(max_depth=3,
                        booster='dart',
                        eta=0.3,
                        silent=1,
                        n_estimators=100)


# Train XGBoost Classifer
booster.fit(x_train,y_train)

modelName = "AutoLoan_XGBoost"
 

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, 

[16:43:33] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:43:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# Train the Tensorflow Model

In [11]:
x_train.shape[0]

4172

In [12]:
# Train Tensorflow Classified with Kera API 
clf_tens = tf.keras.models.Sequential([tf.keras.layers.Dense(units=18, activation = 'relu',input_shape=(x_train.shape[-1],)),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=16, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                                 tf.keras.layers.Dense(units=8, activation = 'relu'),
                                 tf.keras.layers.Dropout(0.9),
                               
                                 # use sigmoid activation for binary target, softmax did not work
                                 tf.keras.layers.Dense(1, activation = 'sigmoid')])

# use binary_crossentropy loss function for binary target
clf_tens.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0006),
              loss='binary_crossentropy',
              metrics = ['acc'])

# train a model
clf_tens.fit(x_train,y_train, epochs=500,batch_size=int(len(x_train)/10),verbose=1,validation_data=(x_test, y_test))

modelName2 = "AutoLoan_Tensorflow"

2022-06-10 16:43:34.710640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-10 16:43:34.710679: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-10 16:43:34.710704: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eeclxvm068.exnet.sas.com): /proc/driver/nvidia/version does not exist
2022-06-10 16:43:34.711060: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/keras/optimizer_v2/rmsp

Epoch 1/500
11/11 [==============================] - 1s 18ms/step - loss: 1.9661 - acc: 0.7733 - val_loss: 0.7291 - val_acc: 0.3949
Epoch 2/500
11/11 [==============================] - 0s 4ms/step - loss: 1.9318 - acc: 0.7735 - val_loss: 0.7195 - val_acc: 0.4396
Epoch 3/500
11/11 [==============================] - 0s 4ms/step - loss: 2.0878 - acc: 0.7766 - val_loss: 0.7109 - val_acc: 0.4815
Epoch 4/500
11/11 [==============================] - 0s 4ms/step - loss: 1.8522 - acc: 0.7790 - val_loss: 0.7047 - val_acc: 0.5134
Epoch 5/500
11/11 [==============================] - 0s 4ms/step - loss: 1.7912 - acc: 0.7699 - val_loss: 0.6979 - val_acc: 0.5481
Epoch 6/500
11/11 [==============================] - 0s 4ms/step - loss: 2.1124 - acc: 0.7701 - val_loss: 0.6917 - val_acc: 0.5878
Epoch 7/500
11/11 [==============================] - 0s 4ms/step - loss: 1.9488 - acc: 0.7675 - val_loss: 0.6856 - val_acc: 0.6370
Epoch 8/500
11/11 [==============================] - 0s 4ms/step - loss: 1.6674 - 

11/11 [==============================] - 0s 4ms/step - loss: 0.6288 - acc: 0.8006 - val_loss: 0.5713 - val_acc: 0.7919
Epoch 64/500
11/11 [==============================] - 0s 3ms/step - loss: 0.6359 - acc: 0.8001 - val_loss: 0.5698 - val_acc: 0.7919
Epoch 65/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6221 - acc: 0.8008 - val_loss: 0.5689 - val_acc: 0.7919
Epoch 66/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6473 - acc: 0.7991 - val_loss: 0.5678 - val_acc: 0.7919
Epoch 67/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6699 - acc: 0.7987 - val_loss: 0.5670 - val_acc: 0.7919
Epoch 68/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6715 - acc: 0.8015 - val_loss: 0.5660 - val_acc: 0.7919
Epoch 69/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6623 - acc: 0.7989 - val_loss: 0.5650 - val_acc: 0.7919
Epoch 70/500
11/11 [==============================] - 0s 4ms/step - loss: 0.6210 - acc: 0

Epoch 125/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5296 - acc: 0.8042 - val_loss: 0.5271 - val_acc: 0.7919
Epoch 126/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5417 - acc: 0.8035 - val_loss: 0.5268 - val_acc: 0.7919
Epoch 127/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5302 - acc: 0.8032 - val_loss: 0.5263 - val_acc: 0.7919
Epoch 128/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5377 - acc: 0.8032 - val_loss: 0.5262 - val_acc: 0.7919
Epoch 129/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5471 - acc: 0.8032 - val_loss: 0.5262 - val_acc: 0.7919
Epoch 130/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5353 - acc: 0.8030 - val_loss: 0.5260 - val_acc: 0.7919
Epoch 131/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5345 - acc: 0.8039 - val_loss: 0.5258 - val_acc: 0.7919
Epoch 132/500
11/11 [==============================] - 0s 4ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.5162 - acc: 0.8039 - val_loss: 0.5144 - val_acc: 0.7919
Epoch 187/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5137 - acc: 0.8039 - val_loss: 0.5143 - val_acc: 0.7919
Epoch 188/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5107 - acc: 0.8047 - val_loss: 0.5142 - val_acc: 0.7919
Epoch 189/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5051 - acc: 0.8044 - val_loss: 0.5141 - val_acc: 0.7919
Epoch 190/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5117 - acc: 0.8042 - val_loss: 0.5140 - val_acc: 0.7919
Epoch 191/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5068 - acc: 0.8039 - val_loss: 0.5139 - val_acc: 0.7919
Epoch 192/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5055 - acc: 0.8042 - val_loss: 0.5137 - val_acc: 0.7919
Epoch 193/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5082 -

Epoch 248/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4990 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 249/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5005 - acc: 0.8039 - val_loss: 0.5119 - val_acc: 0.7919
Epoch 250/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4995 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 251/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4989 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 252/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5004 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 253/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5040 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 254/500
11/11 [==============================] - 0s 4ms/step - loss: 0.5012 - acc: 0.8042 - val_loss: 0.5118 - val_acc: 0.7919
Epoch 255/500
11/11 [==============================] - 0s 4ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.4944 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 310/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4958 - acc: 0.8039 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 311/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4954 - acc: 0.8039 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 312/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4981 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 313/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4962 - acc: 0.8044 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 314/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4971 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 315/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4975 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 316/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4959 -

Epoch 371/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4957 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 372/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4943 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 373/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4941 - acc: 0.8042 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 374/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4937 - acc: 0.8039 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 375/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4953 - acc: 0.8044 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 376/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4960 - acc: 0.8042 - val_loss: 0.5115 - val_acc: 0.7919
Epoch 377/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4936 - acc: 0.8042 - val_loss: 0.5114 - val_acc: 0.7919
Epoch 378/500
11/11 [==============================] - 0s 4ms/step - 

11/11 [==============================] - 0s 4ms/step - loss: 0.4954 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 433/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4961 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 434/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4940 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 435/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4949 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 436/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4961 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 437/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4941 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 438/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4952 - acc: 0.8042 - val_loss: 0.5112 - val_acc: 0.7919
Epoch 439/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4950 -

Epoch 494/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4954 - acc: 0.8042 - val_loss: 0.5108 - val_acc: 0.7919
Epoch 495/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4953 - acc: 0.8042 - val_loss: 0.5108 - val_acc: 0.7919
Epoch 496/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4962 - acc: 0.8042 - val_loss: 0.5108 - val_acc: 0.7919
Epoch 497/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4933 - acc: 0.8042 - val_loss: 0.5108 - val_acc: 0.7919
Epoch 498/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4934 - acc: 0.8042 - val_loss: 0.5108 - val_acc: 0.7919
Epoch 499/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4951 - acc: 0.8042 - val_loss: 0.5109 - val_acc: 0.7919
Epoch 500/500
11/11 [==============================] - 0s 4ms/step - loss: 0.4930 - acc: 0.8042 - val_loss: 0.5107 - val_acc: 0.7919


# Pickle the trained XGB Model and H5 for Tensorflow Model

In [13]:
#pickle.dump(model, open(project_dir+"/HMEQ_XGB.pickle", 'wb'))
pickle.dump(booster, open("/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName + ".pickle", 'wb'))
pickle.dump(booster, open("Data_orig"+"/"+ modelName + ".pickle", 'wb'))


clf_tens.save(filepath = "/sasinside/userdata/gegrab/resources/hmeq" + '/' + modelName2 + '.h5')

# Write the XGBoost Model to ONNX


In [14]:
model     = booster
modelName3 = "AutoLoan_ONNX_XGBoost"

column_count = x_train.shape[1] #second position
column_count



18

In [15]:
import onnx
from onnxmltools.convert.common import data_types

initial_type = [('float_input', data_types.FloatTensorType([1, column_count]))]

In [16]:
from onnxmltools.convert import convert_xgboost

booster_onnx = convert_xgboost(booster, initial_types=initial_type)
onnx.save(booster_onnx, "/sasinside/userdata/gegrab/resources/hmeq"+"/"+ modelName3 +".onnx")
 

# Write Out Parameters for Copying

In [17]:
# Define input variables and perform formatting to match with score code file

input_params = ''
for col in auto.columns:
    input_params += col
    if col != auto.columns[-1]:
        input_params += ', '

input_cols = ''
for col in auto.columns:
    input_cols += "\"" + col + "\""
    if col != auto.columns[-1]:
        input_cols += ', '


print(input_cols)
print("")
print(input_params)

"Loan_ID", "BAD", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"

Loan_ID, BAD, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC


# Test ONNX Scoring Code

In [18]:
def scoreONNX(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    

    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_BAD = float(pred_onnx[0][0])
    
    
     
    return (P_BAD)


 

In [19]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/autoloan2_ds2_test.csv') 
X_test2 
 
# gene2 = gene.drop(['BAD','Loan_ID'], axis=1)
 
for i in range(5):
    print(scoreONNX(**X_test2.iloc[i]))

0.050464928150177
0.09749710559844971
0.03160446882247925
0.1253322958946228
0.23529398441314697


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/950844807.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/950844807.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

# Write Out ONNX Score Code

In [20]:
score_code = """
def scoreONNX(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    
    import onnxruntime as rt
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    

    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = rt.InferenceSession("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_ONNX_XGBoost.onnx")
    
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    probabilities = sess.get_outputs()[1].name
    
    for x in test_data:
        pred_onnx = sess.run([probabilities], {input_name: np.expand_dims(x, axis=0).astype(np.float32)})[0]
    

    P_BAD = float(pred_onnx[0][0])
    
    
     
    return (P_BAD)"""



f = open('Data_orig/AutoLoan_ONNX_XGB.py',"w+")
f.write(score_code)
f.close()

# Test XGBoost Score Code

In [21]:
def scoreXGB(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_XGBoost.pickle", 'rb'))

    P_BAD = float(sess.predict_proba(test_data)[0,0])
    
    
     
    
     
     
    return (P_BAD)



In [22]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/autoloan2_ds2_test.csv') 
X_test2 
 
 
 
for i in range(5):
    print(scoreXGB(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/948797846.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.05046498775482178
0.09749722480773926
0.03160446882247925
0.1253322958946228
0.23529398441314697


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/948797846.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/948797846.py:25: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

# Write Out XGBoost Score Code

In [23]:
score_code = """
def scoreXGB(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle")
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
     
    sess = pickle.load(open("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_XGBoost.pickle", 'rb'))

    P_BAD = float(sess.predict_proba(test_data)[0,0])
    
    
     
   
    return (P_BAD)"""


f = open('Data_orig/AutoLoan_XGB.py',"w+")
f.write(score_code)
f.close()

# Test Tensorflow Score Code

In [24]:
def scoreTF(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC ):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
     
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_Tensorflow.h5")
    
    P_BAD = float(sess.predict(test_data))
    P_BAD = float(tf.keras.activations.sigmoid(P_BAD))
     
    
    return (P_BAD)



In [25]:
import pandas as pd

X_test2 = pd.read_csv('Data_orig/autoloan2_ds2_test.csv') 
X_test2 
 
 
 
for i in range(5):
    print(scoreTF(**X_test2.iloc[i]))

/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/629741888.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)


0.5489851236343384
0.5489851236343384


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/629741888.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/629741888.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.5489851236343384
0.5489851236343384


/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/629741888.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(categorical_cols,1,inplace=True)
/sasinside/miniconda3/envs/pytorch2021/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/tmp/ipykernel_30379/629741888.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

0.5484648942947388


# Write Out Tensorflow Score Code

In [26]:
score_code = """
def scoreTF(Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC ):
    "Output: P_BAD"
        
    
    import pickle
    import numpy as np
    import pandas as pd
    import tensorflow as tf
    
    inputArray = pd.DataFrame([[Loan_ID, LOAN, LOANDUE, VALUE, REASON, JOB, YOJ, DEROG, DELINQ, CLAGE, NINQ, CLNO, DEBTINC]],
                              columns = ["Loan_ID", "LOAN", "LOANDUE", "VALUE", "REASON", "JOB", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
                               )
 

    def preprocessing(df, ohe_loc = None):

        categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
    
        with open(ohe_loc, "rb") as ohe_file:
            ohe = pickle.load(ohe_file) 
        
        enc = ohe.transform(df[categorical_cols])
    
        df[ohe.get_feature_names(categorical_cols).tolist()] = pd.DataFrame(enc, index=df.index)

        df.drop(categorical_cols,1,inplace=True)

        df.dropna(inplace=True)
    
        return df

    
    inputArray.fillna(pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_impute.pickle', 'rb')),inplace=True)
    inputArray.REASON.replace(np.nan,'HomeImp',regex = True, inplace=True)
    inputArray.JOB.replace(np.nan,'Other',regex = True, inplace=True)
    
    inputArray.drop(['Loan_ID'], axis=1, inplace=True)
    test_data = preprocessing(inputArray, "/sasinside/userdata/gegrab/resources/hmeq/autoloan_encoder.pickle") 
    
    scaler = pickle.load(open('/sasinside/userdata/gegrab/resources/hmeq/autoloan_scaler.pickle', 'rb'))    
    
    test_data = scaler.transform(test_data)    
    
     
     
    sess = tf.keras.models.load_model("/sasinside/userdata/gegrab/resources/hmeq/AutoLoan_Tensorflow.h5")
    
    P_BAD = float(sess.predict(test_data))
    P_BAD = float(tf.keras.activations.sigmoid(P_BAD))
     
    
    return (P_BAD)"""


f = open('Data_orig/AutoLoan_Tensorflow.py',"w+")
f.write(score_code)
f.close()


